# Lab Two
### Authors: Eric Smith, Tyler Giallanza, Oscar Vallner, Momin Irfan

#### Special thanks to [Justin Ledford](https://github.com/justinledford) for a lot of the code.

_________

## Unpooling

We implemented unpooling by creating our own custom layers: MaxPoolingMax2D and Unpooling. The MaxPoolingMask2D is used with our encoder to save the locations of the maximums when the image is pooled whilst encoded. The unpooling takes the encoded image and the masks which it then unpools.

![Unpooling](./output_images/unpool.PNG)

The above image shows the strategy we employed to implement unpooling.The unpooling custom layers are included below and can be accessed 

In [ ]:
from keras import backend as K
from keras.layers.convolutional import UpSampling2D
from keras.layers import MaxPooling2D, Layer
import tensorflow as tf

class MaxPoolingMask2D(MaxPooling2D):
    def __init__(self, pool_size=(2, 2), strides=None, **kwargs):
        super(MaxPoolingMask2D, self).__init__(pool_size, strides, **kwargs)

    def _pooling_function(self, inputs, pool_size, strides, padding, data_format):
        pooled = K.pool2d(inputs, pool_size, strides, pool_mode='max')
        upsampled = UpSampling2D(size=pool_size)(pooled)
        indexMask = K.tf.equal(inputs, upsampled)
        assert indexMask.get_shape().as_list() == inputs.get_shape().as_list()
        return indexMask

    def get_output_shape_for(self, input_shape):
        return input_shape

class Unpooling(Layer):
    def __init__(self, **kwargs):
        super(Unpooling, self).__init__(**kwargs)

    def build(self, input_shape):
        pass

    def call(self, x, mask=None):
        layer = x[0]
        layer_mask = x[1]
        print('unpooling input - layer shape:',layer.shape,'mask shape:',layer_mask.shape)
        mask_shape = layer_mask.get_shape().as_list()
        layer_shape = layer.get_shape().as_list()
        pool_size = (2,2) 

        on_success = tf.keras.backend.resize_images(layer, 2, 2, 'channels_last')

        on_fail = K.zeros_like(on_success)
        output =  K.tf.where(K.tf.cast(layer_mask,bool), on_success, on_fail)
        print('    ',output.shape)
        return output


    def compute_output_shape(self, input_shape):
        return input_shape[1]



_________

## Image Reconstruction
We constructed a few images based using the new trained decoders.

| Layer | Output |
|---|---|
| Original: |  ![doge-256](./input_images/doge-256.jpg)|
| Layer 2: | ![doge-256-2](./output_images/doge-decoded_2.jpg)|
| Layer 3: | ![doge-256-2](./output_images/doge-decoded_3.jpg)|
| Layer 4: | ![doge-256-2](./output_images/doge-decoded_4.jpg)|
| Layer 5: | ![doge-256-2](./output_images/doge-decoded_5.jpg)|

----

## Styling

|Content|Style|Output|
|---|---|---|
|![content](./input_images/doge-256.jpg)|![style](./input_images/wavestyle.jpg)|![output](./output_images/wavestyled_doge.jpg)|